# Define Audit Process

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [8]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [11]:
select top 10 * 
from adlste_coa.coa_audit_summary 
order by 1 desc

Run_ID,Site_ID,LogDate,TableName,CallingSP,SQLVersion,Run_Type,Steps,Errors,Warnings,Start_TS,End_TS
2179,TRANSCEND02,2020-09-29,adlste_coa.coat_dat_top_users,adlste_coa.sp_dat_top_users,v1,normal run,2,0,0,2020-09-29 06:39:13,2020-09-29 06:39:15
2177,TRANSCEND02,2020-09-29,adlste_coa.coat_dat_feature_usage_log,adlste_coa.sp_dat_feature_usage_log,v1,normal run,2,0,0,2020-09-29 06:38:49,2020-09-29 06:38:51
2175,TRANSCEN,2020-09-29,adlste_coa.coat_dat_permspace_week,adlste_coa.sp_dat_permspace_week,v1,normal run,2,0,0,2020-09-29 06:38:40,2020-09-29 06:38:41
2171,TRANSCEND02,2020-09-29,adlste_coa.coat_dat_DBQL_Core_QryCnt_Ranges,adlste_coa.sp_dat_DBQL_Core_QryCnt_Ranges,6.0,normal run,4,0,0,2020-09-29 06:38:27,2020-09-29 06:38:28
2167,TRANSCEND02,2020-09-29,adlste_coa.coat_dat_DBQL_core,adlste_coa.sp_dat_DBQL_core,6.0,normal run,4,0,0,2020-09-29 06:38:19,2020-09-29 06:38:25
2165,TRANSCEND02,2020-09-29,adlste_coa.coat_dat_DBQL_Core_MaxCPU,adlste_coa.sp_dat_DBQL_Core_MaxCPU,6.0,normal run,2,0,0,2020-09-29 06:36:14,2020-09-29 06:36:15
2163,TRANSCEN,2020-09-29,adlste_coa.coat_dat_permspace_week,adlste_coa.sp_dat_permspace_week,v1,normal run,2,0,0,2020-09-29 06:36:12,2020-09-29 06:36:13
2161,TRANSCEN,2020-09-29,adlste_coa.coat_dat_permspace_week,adlste_coa.sp_dat_permspace_week,v1,normal run,2,0,0,2020-09-29 06:36:02,2020-09-29 06:36:03
2157,AAPROD4,2020-09-29,adlste_coa.CB_DAILY_SUMMARY_CURR_PREV,adlste_coa.sp_dat_CB_DAILY_SUMMARY_CURR_PREV,0.6,normal run,4,0,0,2020-09-29 01:37:07,2020-09-29 01:37:08
2153,AAPROD4,2020-09-28,adlste_coa.CB_DAILY_SUMMARY_CURR_PREV,adlste_coa.sp_dat_CB_DAILY_SUMMARY_CURR_PREV,0.6,normal run,4,0,0,2020-09-28 22:55:28,2020-09-28 22:55:28


In [ ]:
select top 10 * from adlste_coa.coa_audit_summary order by 1 desc

In [ ]:
call adlste_coa.sp_meta_objects_refresh('none');

In [ ]:
Select * from adlste_coa.coa_meta_objects
where Tablename  like '%mat%'
order by ObjType, ProcessName,  ObjType desc;

<br/><br/><br/>
## Audit_Run (sub-stored_procedure)
### for tracking audit columns per row while minimizing storage
- database:    adlste_coa
- table:       coat_audit_run
- view:        coa_audit_run
- stored proc: sp_audit_run

In [ ]:
drop table adlste_coa.coat_audit_runlog

In [ ]:
Create multiset table adlste_coa.coat_audit_runlog
(
 Run_ID       INTEGER 
,ParentRun_ID INTEGER
,Site_ID      VARCHAR(128)
,TableName    VARCHAR(128)
,StartDate    DATE
,EndDate      DATE
,CallingSP    VARCHAR(128)
,SQLVersion   VARCHAR(128)
,Run_Type     VARCHAR(128)
,Run_Status   VARCHAR(128)
,Run_Message  VARCHAR(512)
,Run_User     VARCHAR(128) Default USER
,Run_TS       Timestamp(0) Default Current_Timestamp(0)
) 
primary index (Site_ID, TableName);

In [ ]:
replace view adlste_coa.coa_audit_runlog 
as locking row for access
Select * from adlste_coa.coat_audit_runlog

In [ ]:
replace PROCEDURE adlste_coa.sp_audit_log (inout runid integer
                                              ,in siteid varchar(128)
                                              ,in tblname varchar(128)
                                              ,in callingsp varchar(128)
                                              ,in sqlversion varchar(128)
                                              ,in startdate date
                                              ,in enddate date
                                              ,in runtype varchar(128)
                                              ,in runstatus varchar(128)
                                              ,in runmessage varchar(512)
                                              )
SQL SECURITY CREATOR
BEGIN
DECLARE runidout INTEGER;

    SELECT coalesce(Max(Run_ID)+1,1) as Run_ID   INTO runidout 
    FROM adlste_coa.coat_audit_runlog 
    ;
    
    Insert into adlste_coa.coat_audit_runlog 
    values
    ( :runidout
     ,:runid --parentrunid
     ,:siteid
     ,:tblname
     ,:startdate
     ,:enddate
     ,:callingsp
     ,:sqlversion
     ,:runtype
     ,:runstatus
     ,:runmessage
     ,USER 
     ,Current_Timestamp(0)
    );
    
    SET runid = runidout ;

END;

In [ ]:
-- delete from adlste_coa.coat_audit_runlog -- don't do this

### Example of some calls - designed to be embedded in other SPs

In [ ]:
-- start SP()
CALL adlste_coa.sp_audit_log(0
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'START' );  -- Run Status

In [ ]:
-- do stuff, but need a checkpoint 
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'CHECKPOINT' );  -- Run Status

In [ ]:
-- end of SP()
-- (number above becomes last parameter below)
call adlste_coa.sp_audit_log(4
                            ,'test_SiteID'  -- Site_ID
                            ,'adlste_coa.coat_audit_run'  -- Table Name
                            ,'adlste_coa.sp_test'  -- Calling SP
                            ,'v1.15'  -- StoredProc Version
                            ,DATE-7 -- Start Date
                            ,DATE-1 -- End Date
                            ,'Test Run'  -- Run Type
                            ,'END'); -- Run Status

In [ ]:
replace view adlste_coa.coa_audit_log as
locking row for access
Select * from adlste_coa.coat_audit_runlog

In [ ]:
replace view adlste_coa.coa_audit_summary   as locking row for access
Select 
 case when ParentRun_ID=0 then Run_ID else ParentRun_ID end as Run_ID
,Site_ID
,min(cast(Run_TS as DATE)) as LogDate
,TableName
,CallingSP
,SQLVersion
,Run_Type
,count(*) as Steps
,sum(case when Run_Status='ERROR' then 1 else 0 end) as Errors
,sum(case when Run_Status='WARNING' then 1 else 0 end) as Warnings
,min(case when Run_Status='START' then Run_TS else NULL end) as Start_TS
,max(case when Run_Status='END' then Run_TS else NULL end) as End_TS
from adlste_coa.coa_audit_runlog
Group by 1,2,4,5,6,7

In [ ]:
replace view adlste_coa.coa_audit_log_summary as
select * from adlste_coa.coa_audit_summary 

In [ ]:
select * from adlste_coa.coa_audit_log order  by 1 desc

In [ ]:
select * from adlste_coa.coa_audit_summary order  by 1 desc